<img src="https://www.pucsp.br/sites/default/files/download/brasao-PUCSP-assinatura-principal-RGB.png" alt="Logo PUC / ODS" style="width: 20%;">

# Otimização Nutricional
### O objetivo do projeto é desenvolver um software que utilize programação linear na otimização das quantidades necessárias de alimentos, com o objetivo de atender às necessidades nutricionais de um indivíduo adulto. A atividade começará com uma análise de uma planilha fornecida, contendo informações nutricionais de diversos alimentos

### Grupo

| Nome | 
| --- | 
| Ana Beatriz Pinheiro | 
| Ana Clara Rodrigues |
| Ana Paula Moura |
| Lívia Rainho |
| Renan Teano |


### Bibliotecas

In [3]:
#importando todas as bibliotecas necessárias
!pip install pulp
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np

### Data Frames

In [4]:
#Criando um Data Frame 
df_alimentos = pd.read_excel("Informações Nutricionais - P1 O&S.xlsx")

# Manipulando os dados 
colunas_transformar = ['Energia (kcal)', 'Proteina (g)', 'Lipideos (g)', 'Carboidrato (g)', 
                       'Calcio (mg)', 'Ferro (mg)', 'Vitamina A (mg)', 'Vitamina C (mg)']

for coluna in colunas_transformar:
    # Transformar todos os dados em numerico
    df_alimentos[coluna] = pd.to_numeric(df_alimentos[coluna], errors='coerce')
    
    # Colocando os valores Nan em 0
    df_alimentos[coluna] = df_alimentos[coluna].fillna(0)
    
    # Convertendo os valores para inteiro
    df_alimentos[coluna] = df_alimentos[coluna].astype(int)

df_alimentos.head()



,Id,Nome,Período,Categoria,Umidade (%),Energia (kcal),Proteina (g),Lipideos (g),Carboidrato (g),Calcio (mg),Ferro (mg),Vitamina A (mg),Vitamina C (mg)
0,1,"Arroz, integral, cozido",Jantar,Carboidrato,70.138667,123,2,1,25,5,0,0,0
1,2,"Arroz, cozido",Almoço,Carboidrato,69.113667,128,2,0,28,3,0,0,0
2,3,"Biscoito, doce, maisena",Lanche,Carboidrato,3.2,442,8,11,75,54,1,0,6
3,4,"Biscoito, salgado, cream cracker",Lanche,Carboidrato,4.056667,431,10,14,68,20,2,0,0
4,5,"Bolo, pronto, chocolate",Lanche,Carboidrato,19.276333,410,6,18,54,74,2,0,0


In [5]:
# Criando um Data Frame com o arquivo de necessidades nutricionais 
df_necessidades = pd.read_excel("Quantidade Necessária.xlsx")
df_necessidades.head()

,Sexo,Peso (kg),Proteina (g),Carboidrato (g),Lipideos (g),Vitamina A (mg),Vitamina C (mg),Calcio (mg),Ferro (mg)
0,Homem,50,40,200,50,900,90,1000,8
1,Homem,60,48,240,60,900,90,1000,8
2,Homem,70,56,280,70,900,90,1000,8
3,Homem,80,64,320,80,900,90,1000,8
4,Homem,90,72,360,90,900,90,1000,8


### Listas

In [6]:
#Cração de 3 listas vazias
cafe = []
almoco = []
jantar = []
lanche = []
#loop para iterar as linhas do DataFrame
for i, row in df_alimentos.iterrows():
    periodo = row["Período"]
    nome = row["Nome"]
    nutrientes = {
        'Proteina (g)': row['Proteina (g)'],
        'Lipideos (g)': row['Lipideos (g)'],
        'Carboidrato (g)': row['Carboidrato (g)'],
        'Calcio (mg)': row['Calcio (mg)'],
        'Ferro (mg)': row['Ferro (mg)'],
        'Vitamina A (mg)': row['Vitamina A (mg)'],
        'Vitamina C (mg)': row['Vitamina C (mg)']
    }
#Condições para criar a lista conforme os dados da coluna periodo
    if periodo == "Café da Manhã":
        cafe.append(i)
    if periodo == "Lanche":
        lanche.append(i)
    if periodo == "Almoço":
        almoco.append(i)
    if periodo == "Jantar":
        jantar.append(i)
print("Lista de indice dos alimentos para o Café da Manhã:")
print(cafe)
print("Lista de indice dos alimentos para o Lanche:")
print(lanche)
print("Lista de indice dos alimentos para o Almoço:")
print(almoco)
print("Lista de indice dos alimentos para o Jantar:")
print(jantar)

Lista de indice dos alimentos para o Café da Manhã:
[6, 9, 10, 11, 12, 18, 48, 114, 115, 116, 117, 118, 119, 149, 203, 210, 211, 217, 218, 219, 220, 221, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 237, 238, 239, 240, 246, 247, 248, 249, 259, 260, 261, 262, 263, 264, 267]
Lista de indice dos alimentos para o Lanche:
[2, 3, 4, 5, 13, 14, 15, 16, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 180, 212, 213, 222, 257, 258, 293, 294, 295, 296, 297, 298, 299, 300, 301]
Lista de indice dos alimentos para o Almoço:
[1, 8, 17, 19, 22, 23, 24, 26, 27, 41, 43, 44, 45, 46, 47, 49, 52, 53, 54, 55, 56, 57, 112, 113, 120, 121, 122, 123, 126, 127, 128, 135, 136, 137, 138, 139, 145, 146, 147, 148, 155, 156, 157, 158, 159, 160, 162, 163, 164, 165, 166, 170, 173, 183, 184, 185, 186, 187, 188, 189, 190, 20

### Regressão Linear

In [7]:
Mulher = df_necessidades[df_necessidades["Sexo"] == "Mulher"]
x = Mulher["Peso (kg)"]
x = np.array(x).reshape(-1,1)
x

array([[50],
       [55],
       [60],
       [65],
       [70]], dtype=int64)

In [8]:
Mulher.columns

Index(['Sexo', 'Peso (kg)', 'Proteina (g)', 'Carboidrato (g)', 'Lipideos (g)',
       'Vitamina A (mg)', 'Vitamina C (mg)', 'Calcio (mg)', 'Ferro (mg)'],
      dtype='object')

In [9]:
y = Mulher["Proteina (g)"]
y = np.array(y).reshape(-1,1)
y

array([[40],
       [43],
       [46],
       [49],
       [52]], dtype=int64)

In [10]:
#Criando a regressão linear 
model = LinearRegression()
model.fit(x,y)

LinearRegression()

In [11]:
entrada = ["Peso (kg)"]
saidas = ['Proteina (g)', 'Carboidrato (g)', 'Lipideos (g)', 'Vitamina A (mg)', 'Vitamina C (mg)', 'Calcio (mg)', 'Ferro (mg)']
sexo = "Mulher"
peso = 62

dados_filtrados = df_necessidades[df_necessidades["Sexo"] == sexo]

models = {}
for saida in saidas:
    model = LinearRegression()
    model.fit(dados_filtrados[entrada],dados_filtrados[saida],sample_weight=None)
    models[saida]= model

predictions = {}
for saida, model in models.items():
    prediction = model.predict(pd.DataFrame(data = [[peso]], columns = entrada))
    predictions[saida] = prediction[0]
    
print(f"Valores interpolados para {sexo} com {peso} kg:")
for target, value in predictions.items():
    print(f"{target}: {value:.0f}")

Valores interpolados para Mulher com 62 kg:
Proteina (g): 47
Carboidrato (g): 236
Lipideos (g): 62
Vitamina A (mg): 700
Vitamina C (mg): 75
Calcio (mg): 1000
Ferro (mg): 18


### Otimização

In [12]:
from pulp import LpVariable, LpProblem, lpSum, LpMinimize, LpStatus
nutriente_necessário = {
    "Proteina (g)": 46,
    "Carboidrato (g)": 230,
    "Lipideos (g)": 60,
    "Vitamina A (mg)": 700,
    "Vitamina C (mg)": 75,
    "Calcio (mg)": 1000,
    "Ferro (mg)": 18
}
alimentos_selecionados = []

def otimizar():
    global alimentos_selecionados
    #Criação do problema
    prob = LpProblem("Dieta", LpMinimize)
    #Criação das variáveis
    quantidade = LpVariable.dicts("Quantidade", df_alimentos.index, lowBound=0, upBound=1, cat = 'Integer')
    #objetivo: minimizar calorias totais
    prob += lpSum([df_alimentos.loc[i, "Energia (kcal)"] * quantidade[i] for i in df_alimentos.index])
    #restrição principal
    for nutriente, quant in nutriente_necessário.items():
        prob += lpSum([df_alimentos.loc[j, nutriente] * quantidade[j] for j in df_alimentos.index]) >= quant
    
    #restrições
    prob += lpSum([quantidade[i]for i in cafe]) == 4
    prob += lpSum([quantidade[i]for i in lanche]) == 3
    prob += lpSum([quantidade[i]for i in almoco]) == 5
    prob += lpSum([quantidade[i]for i in jantar]) == 5
    #restringir para que não venha alimentos repetidos
    prob += lpSum([quantidade[i]for i in alimentos_selecionados]) == 0
    #resolução
    prob.solve()

    alimentos_selecionados.extend([i for i in df_alimentos.index if quantidade[i].varValue > 0])
    
    print("Status:", LpStatus[prob.status])
    if LpStatus[prob.status] == "Optimal":
        print("Solução Ótima Encontrada:")
        
        print("\nRefeição 1 - Café da Manhã:")
        alimentos_cafe = [i for i in cafe if quantidade[i].varValue > 0]
        for i in alimentos_cafe[:4]:
            print(f"{df_alimentos.loc[i, 'Nome']} = {quantidade[i].varValue}")

        print("\nRefeição 2 - Lanche:")
        alimentos_lanche = [i for i in lanche if quantidade[i].varValue > 0]
        for i in alimentos_lanche[:4]:
            print(f"{df_alimentos.loc[i,'Nome']} = {quantidade[i].varValue}")

        print("\nRefeição 3 - Almoço:")
        alimentos_almoco = [i for i in almoco if quantidade[i].varValue > 0]
        for i in alimentos_almoco[:4]:
            print(f"{df_alimentos.loc[i,'Nome']} = {quantidade[i].varValue}")

        print("\nRefeição 4 - Jantar:")
        alimentos_jantar = [i for i in jantar if quantidade[i].varValue > 0]
        for i in alimentos_jantar[:4]:
            print(f"{df_alimentos.loc[i,'Nome']} = {quantidade[i].varValue}")
            
    else:
        print("Não foi possível encontrar uma solução viável.")

    # Cálculo do total de calorias
    total_calories = sum([df_alimentos.loc[i, "Energia (kcal)"] * quantidade[idx].varValue for idx in df_alimentos.index if quantidade[idx].varValue > 0])
    print(f"Total Calories: {total_calories:.2f}")

otimizar()



Status: Optimal
Solução Ótima Encontrada:

Refeição 1 - Café da Manhã:
Mingau tradicional, pó = 1.0
Iogurte, sabor abacaxi = 1.0
Chá, erva-doce, infusão 5% = 1.0
Chá, mate, infusão 5% = 1.0

Refeição 2 - Lanche:
Abacate, cru = 1.0
Maracujá, cru = 1.0
Pêra, Williams, crua = 1.0

Refeição 3 - Almoço:
Espinafre, Nova Zelândia, refogado = 1.0
Pepino, cru = 1.0
Tomate, com semente, cru = 1.0
Feijão tropeiro mineiro = 1.0

Refeição 4 - Jantar:
Alface, lisa, crua = 1.0
Repolho, roxo, refogado = 1.0
Rúcula, crua = 1.0
Corvina grande, cozida = 1.0
Total Calories: 1700.00
